##### 第一模块(导入包模块：必须运行）

In [1]:
### 引入所有包,如果缺少某个包，包的名字附于之后
import numpy as np
# numpy

import pandas as pd
# pandas

import requests
# requests

from bs4 import BeautifulSoup
# BeautifulSoup

import re
# re

from fake_useragent import UserAgent
# fake-useragent

import json
# json

import time
# time

import random
# random

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# selenium

from PIL import Image,ImageEnhance
# PIL

import hashlib
# hashlib

from collections import Counter
# collections

from gensim import corpora, models, similarities
# gensim 

import codecs, sys
# codecs

import os
# os

import shutil
# shutil

import jieba
# jieba

from pyhanlp import *
# pyhanlp，注意hanlp需要java的工具

import jpype
# jpype

import tensorflow as tf
# tensorflow

import matplotlib.pyplot as plt
# matplotlib

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer
# sklearn

from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfdevice import PDFDevice
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
# pdfminer3k

from tkinter import *
from tkinter import filedialog
# tk文件导入模块

from functools import reduce
# reduce

##### 第二模块(文件处理模块：必须运行）

In [2]:
def import_table(root):
    """
    return table"选中的表格
    """
    try:
        root.filename = filedialog.askopenfilename(filetypes=(("xlsx", "*.xlsx"),("xlsx", "*.xlsx")))        
        if ".xlsx" in root.filename:
            ### 该目录下有该文件
            table = pd.read_excel(root.filename)
            root.destroy()
            return table
    except Exception as e:
        root.destroy()
        print("导入错误")

def remote_select():
    print("以下文件可以调用，需要分析哪个文件？")
    print("-------------------------------------")
    index = 0
    list_item_temp = []
    for item in os.listdir():
        if ".xlsx" in item:
            index += 1
            print("[" + str(index) + "] " + item)
            list_item_temp.append(item)
    try:
        bash_pos = "/Users/dfuser/Desktop/目标文书目录/"
        file_code = int(input("需要导入哪个文件？(输入[]中的序号)"))
        final_pos = bash_pos + str(list_item_temp[file_code-1])
        table_ = pd.read_excel(final_pos)
        return table_
        print("表格导入成功，以下是表格预览")
        print("----------------------------")
    except Exception as e:
        print("导入错误")
        
def save_model(table):
    print("-------------------------------------------")
    print("以下为表格的预览:")
    print(table[new_table.columns].head())
    flag = input("是否需要保存该词频表？(输入Y/N):")
    try:
        if flag == "Y":
            save_file_name = input("请输入该词频表格的名称(不用加.xlsx):")
            save_file_name_xlsx = save_file_name + ".xlsx"
            table.to_excel(save_file_name_xlsx)
            print("存储完毕")
        else:
            print("未能存储")
            pass
    except Exception as e:
        pass

##### 第三模块(核心提取器模块：必须运行）

In [3]:
# -*- coding:utf-8 -*-
## 查询含有关键词的上下文
import re

# 正当防卫查询示例
list_pattern = []
def generate_pattern():
    expression = input("请输入需要使用的正则表达式:")
    pattern = re.compile(expression)
    list_pattern.append(pattern)
    
def control_generate():
    while True:
        generate_pattern()
        flag_str = input("请问还需要输入吗？（输入Y/N）")
        if flag_str == "Y":
            continue
        else:
            break

def clean_none(elem):
    if elem == []:
        elem.append("None")
    if elem == None:
        elem = ["None"]
    return elem

def find_keywords_in_courts(series,*patterns):
    list_key_expressions,list_combine,list_combine_clean = [],[],[]
    for pattern in patterns:
        for item in series:
            try:
                facts = re.findall(pattern,item)
                list_key_expressions.append(facts)
            except Exception as e:
                facts = None
                list_key_expressions.append(None)
        list_combine.append(list_key_expressions)
        list_key_expressions = []
    for li in list_combine:
        list_key_expressions_clean = list(map(clean_none,li))
        list_combine_clean.append(list_key_expressions_clean)
    return list_combine_clean

def run(series):
    ## 产生正则表达式
    control_generate()
    list_key_expressions = find_keywords_in_courts(series,*list_pattern)
    return list_key_expressions

#### 抽取器运行模块

In [4]:
if __name__ == '__main__':
    flag_input = int(input("你是在本机上操作？还是远程操作？(1:本机操作,2:远程操作)"))
    if flag_input == 1:
        root = Tk() # 实例化TKinter窗口
        root.withdraw() # 隐藏TKinter窗口
        table_ready_to_eat = import_table(root)
    elif flag_input == 2:
        table_ready_to_eat = remote_select()
    else:
        logging.error("加载错误")
    
    print("表格中所有的列:",table_ready_to_eat.columns)
    selected_column = input("请问需要摘出哪一列的上下文？(输入关键词部分，如果不填默认是法院意见):")
    if selected_column == "":
        selected_column = "法院意见"
    corpus = table_ready_to_eat[selected_column]
    
    list_combine_clean = run(corpus)
    new_table = pd.DataFrame(list_combine_clean).T
    new_table_len = len(new_table.columns)
    new_table.columns = list(map(lambda x:str("拆分结果") + str(x),list(range(1,new_table_len + 1))))
    combine_table = table_ready_to_eat.join(new_table)
    save_model(combine_table)

# 几个万能表达式
#########################
# ，.*?正当防卫.*?，
# ([\u4e00-\u9fa5]{1,20})正当防卫([\u4e00-\u9fa5]{1,20})

# 推荐
#########################
# [\u4e00-\u9fa5]+正当防卫[\u4e00-\u9fa5]+

你是在本机上操作？还是远程操作？(1:本机操作,2:远程操作)2
以下文件可以调用，需要分析哪个文件？
-------------------------------------
[1] 夫妻债务 - 原表.xlsx
[2] ex1.xlsx
[3] 校园暴力 - 结构化重构.xlsx
[4] 校园暴力事件 - 原表.xlsx
[5] 正当防卫 - 原表.xlsx
[6] 夫妻债务.xlsx
[7] .~zhaiwu_regex_filter.xlsx.xlsx
[8] 职业打假人 - 原表.xlsx
需要导入哪个文件？(输入[]中的序号)5
表格中所有的列: Index(['标题', '案号', '案件类型', '庭审程序', '案由', '文书类型', '法院', '判决日期', '原告', '被告',
       '第三人', '法官', '审判长', '审判员', '书记员', '头部', '头部2', '当事人', '当事人2', '庭审程序说明',
       '庭审程序说明2', '庭审过程', '庭审过程2', '庭审过程3', '庭审过程4', '庭审过程5', '庭审过程6', '法院意见',
       '法院意见2', '判决结果', '判决结果2', '庭后告知', '庭后告知2', '结尾', '结尾2', '附录', '附录2'],
      dtype='object')
请问需要摘出哪一列的上下文？(输入关键词部分，如果不填默认是法院意见):
请输入需要使用的正则表达式:[\u4e00-\u9fa5]+正当防卫[\u4e00-\u9fa5]+
请问还需要输入吗？（输入Y/N）N
-------------------------------------------
以下为表格的预览:
                                               拆分结果1
0                                             [None]
1  [马文华的行为是否属于正当防卫的问题, 互相斗殴与正当防卫的区别, 而正当防卫的目的是保护国...
2                                             [None]
3           

#### html阅读器运行模块

In [5]:
# 抠出，加css，存成.html文件
def construct_strong(selected_column):#combine_table):
    list_html_items = []
    for extract1,corpus1 in zip(combine_table["拆分结果1"],combine_table[selected_column]):
        for item in extract1:
            if item != "None":
                if item in corpus1:
                    html_item = "<strong>" + item + "</strong>"
                    list_html_items.append(html_item)
    return list_html_items

def construct_dict(selected_column):
    replace_dict = dict()
    list_html_items = construct_strong(selected_column)
    pattern_extract = re.compile(">.*?<")
    for item_html in list_html_items:
        item_clean = re.search(pattern_extract,item_html).group().strip("><")
        replace_dict[item_clean] = item_html
    return replace_dict

def construct_main_body(selected_column):
    list_corpus_replace = []
    replace_dict = construct_dict(selected_column)
    index = 0
    for text in combine_table[selected_column]:
        if isinstance(text,str):
            index += 1
            flag = "<p>------------------这是第" + str(index) + "份文书--------------------</p>"
            list_corpus_replace.append(flag)
            list_corpus_replace.append("<br>")
            rep = dict((re.escape(k), v) for k, v in replace_dict.items())
            pattern = re.compile("|".join(rep.keys()))
            result = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)
            result_html = "<p class='article'>" + result + "</p>"
            list_corpus_replace.append(result_html)
            list_corpus_replace.append("<br><br><br><br>")
        else:
            index += 1
            flag = "<p>------------------这是第" + str(index)  +"份文书--------------------</p>"
            list_corpus_replace.append(flag)
            list_corpus_replace.append("<br>")
            list_corpus_replace.append("None")
            list_corpus_replace.append("<br><br><br><br>")
    return list_corpus_replace

def construct_html():
    selected_column = input("请问需要摘出哪一列的上下文？(输入关键词部分，如果上面选择法院意见，此处也要选择相同，不填默认是法律意见。):")
    if selected_column == "":
        selected_column = "法院意见"
    list_corpus_replace = construct_main_body(selected_column)
    front = """
    <!DOCTYPE html>
    <html style='background:#eee';">
    <head>
        <title></title>
    </head>
        <style>
        body {
        width: 90%;
        max-width: 960px;
        background: #fff;
        margin: 3em auto 0;
        padding-top: 2em;
        border: 1px solid #ddd;
        border-width: 0 1px;
        }
        .article {
        font:300 1em/1.8 Lantinghei SC, Microsoft Yahei, Hiragino Sans GB, Microsoft Sans Serif, WenQuanYi Micro Hei, sans-serif;
        }
        strong {
        color:white;
        background-color:#ff6600;
        font-family:微软雅黑;
        }
        </style>
    <body>
    """
    end = """
    </body>
    </html>
    """
    div_wrapper = "<div style='padding:5% 10%;position:relative'>" + "".join(list_corpus_replace) + "</div>"
    result = front + div_wrapper + end
    html_name = input("请输入导出html的名字：(不需要加.html)")
    html_name_f = html_name + ".html"
    with open(html_name_f,"w",encoding = "utf-8") as f:
        f.write(result)

In [6]:
construct_html()

请问需要摘出哪一列的上下文？(输入关键词部分，如果上面选择法院意见，此处也要选择相同，不填默认是法律意见。):
请输入导出html的名字：(不需要加.html)E1
